In [ ]:
%mkdir dataset
%cd dataset
!gdown 1N93rb_uFqKRZ9naX8CXShFt5RJHOmjZH
!unzip -q rwf-2000.zip

In [ ]:
%cd ..

# Import libraries

In [ ]:
import os
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from tqdm import tqdm

os.environ["TOKENIZERS_PARALLELISM"] = "false"

import warnings

warnings.filterwarnings("ignore")

In [ ]:
import logging

logging.getLogger().setLevel(logging.INFO)
logging.basicConfig(format="%(message)s", level=logging.INFO)
LOGGER = logging.getLogger("Torch-Cls")
seed = 123
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Model

In [ ]:
class InceptionBlock(nn.Module):
    def __init__(
        self,
        input_dim,
        num_outputs_0_0a,
        num_outputs_1_0a,
        num_outputs_1_0b,
        num_outputs_2_0a,
        num_outputs_2_0b,
        num_outputs_3_0b,
        gating=True,
    ):
        super(InceptionBlock, self).__init__()
        self.conv_b0 = STConv3D(input_dim, num_outputs_0_0a, [1, 1, 1])
        self.conv_b1_a = STConv3D(input_dim, num_outputs_1_0a, [1, 1, 1])
        self.conv_b1_b = STConv3D(
            num_outputs_1_0a, num_outputs_1_0b, [3, 3, 3], padding=1, separable=True
        )
        self.conv_b2_a = STConv3D(input_dim, num_outputs_2_0a, [1, 1, 1])
        self.conv_b2_b = STConv3D(
            num_outputs_2_0a, num_outputs_2_0b, [3, 3, 3], padding=1, separable=True
        )
        self.maxpool_b3 = nn.MaxPool3d((3, 3, 3), stride=1, padding=1)
        self.conv_b3_b = STConv3D(input_dim, num_outputs_3_0b, [1, 1, 1])
        self.gating = gating
        self.output_dim = (
            num_outputs_0_0a + num_outputs_1_0b + num_outputs_2_0b + num_outputs_3_0b
        )
        if gating:
            self.gating_b0 = SelfGating(num_outputs_0_0a)
            self.gating_b1 = SelfGating(num_outputs_1_0b)
            self.gating_b2 = SelfGating(num_outputs_2_0b)
            self.gating_b3 = SelfGating(num_outputs_3_0b)

    def forward(self, input):
        """Inception block"""
        b0 = self.conv_b0(input)
        b1 = self.conv_b1_a(input)
        b1 = self.conv_b1_b(b1)
        b2 = self.conv_b2_a(input)
        b2 = self.conv_b2_b(b2)
        b3 = self.maxpool_b3(input)
        b3 = self.conv_b3_b(b3)
        if self.gating:
            b0 = self.gating_b0(b0)
            b1 = self.gating_b1(b1)
            b2 = self.gating_b2(b2)
            b3 = self.gating_b3(b3)
        return torch.cat((b0, b1, b2, b3), dim=1)


class SelfGating(nn.Module):
    def __init__(self, input_dim):
        super(SelfGating, self).__init__()
        self.fc = nn.Linear(input_dim, input_dim)

    def forward(self, input_tensor):
        """Feature gating as used in S3D-G."""
        spatiotemporal_average = torch.mean(input_tensor, dim=[2, 3, 4])
        weights = self.fc(spatiotemporal_average)
        weights = torch.sigmoid(weights)
        return weights[:, :, None, None, None] * input_tensor


class STConv3D(nn.Module):
    def __init__(
        self, input_dim, output_dim, kernel_size, stride=1, padding=0, separable=False
    ):
        super(STConv3D, self).__init__()
        self.separable = separable
        self.relu = nn.ReLU(inplace=True)
        assert len(kernel_size) == 3
        if separable and kernel_size[0] != 1:
            spatial_kernel_size = [1, kernel_size[1], kernel_size[2]]
            temporal_kernel_size = [kernel_size[0], 1, 1]
            if isinstance(stride, list) and len(stride) == 3:
                spatial_stride = [1, stride[1], stride[2]]
                temporal_stride = [stride[0], 1, 1]
            else:
                spatial_stride = [1, stride, stride]
                temporal_stride = [stride, 1, 1]
            if isinstance(padding, list) and len(padding) == 3:
                spatial_padding = [0, padding[1], padding[2]]
                temporal_padding = [padding[0], 0, 0]
            else:
                spatial_padding = [0, padding, padding]
                temporal_padding = [padding, 0, 0]
        if separable:
            self.conv1 = nn.Conv3d(
                input_dim,
                output_dim,
                kernel_size=spatial_kernel_size,
                stride=spatial_stride,
                padding=spatial_padding,
                bias=False,
            )
            self.bn1 = nn.BatchNorm3d(output_dim)
            self.conv2 = nn.Conv3d(
                output_dim,
                output_dim,
                kernel_size=temporal_kernel_size,
                stride=temporal_stride,
                padding=temporal_padding,
                bias=False,
            )
            self.bn2 = nn.BatchNorm3d(output_dim)
        else:
            self.conv1 = nn.Conv3d(
                input_dim,
                output_dim,
                kernel_size=kernel_size,
                stride=stride,
                padding=padding,
                bias=False,
            )
            self.bn1 = nn.BatchNorm3d(output_dim)

    def forward(self, input):
        out = self.relu(self.bn1(self.conv1(input)))
        if self.separable:
            out = self.relu(self.bn2(self.conv2(out)))
        return out


class MaxPool3dTFPadding(nn.Module):
    def __init__(self, kernel_size, stride=None, padding="SAME"):
        super(MaxPool3dTFPadding, self).__init__()
        if padding == "SAME":
            padding_shape = self._get_padding_shape(kernel_size, stride)
            self.padding_shape = padding_shape
            self.pad = nn.ConstantPad3d(padding_shape, 0)
        self.pool = nn.MaxPool3d(kernel_size, stride, ceil_mode=True)

    def _get_padding_shape(self, filter_shape, stride):
        def _pad_top_bottom(filter_dim, stride_val):
            pad_along = max(filter_dim - stride_val, 0)
            pad_top = pad_along // 2
            pad_bottom = pad_along - pad_top
            return pad_top, pad_bottom

        padding_shape = []
        for filter_dim, stride_val in zip(filter_shape, stride):
            pad_top, pad_bottom = _pad_top_bottom(filter_dim, stride_val)
            padding_shape.append(pad_top)
            padding_shape.append(pad_bottom)
        depth_top = padding_shape.pop(0)
        depth_bottom = padding_shape.pop(0)
        padding_shape.append(depth_top)
        padding_shape.append(depth_bottom)
        return tuple(padding_shape)

    def forward(self, inp):
        inp = self.pad(inp)
        out = self.pool(inp)
        return out


class S3D(nn.Module):
    def __init__(
        self, dict_path=None, num_classes=512, gating=True, space_to_depth=True
    ):
        super(S3D, self).__init__()
        self.num_classes = num_classes
        self.gating = gating
        self.space_to_depth = space_to_depth
        if space_to_depth:
            self.conv1 = STConv3D(
                15, 64, [2, 4, 4], stride=1, padding=(1, 2, 2), separable=False
            )
        else:
            self.conv1 = STConv3D(
                3, 64, [3, 7, 7], stride=2, padding=(1, 3, 3), separable=False
            )
        self.conv_2b = STConv3D(64, 64, [1, 1, 1], separable=False)
        self.conv_2c = STConv3D(64, 192, [3, 3, 3], padding=1, separable=True)
        self.gating = SelfGating(192)
        self.maxpool_2a = MaxPool3dTFPadding(
            kernel_size=(1, 3, 3), stride=(1, 2, 2), padding="SAME"
        )
        self.maxpool_3a = MaxPool3dTFPadding(
            kernel_size=(1, 3, 3), stride=(1, 2, 2), padding="SAME"
        )
        self.mixed_3b = InceptionBlock(192, 64, 96, 128, 16, 32, 32)
        self.mixed_3c = InceptionBlock(
            self.mixed_3b.output_dim, 128, 128, 192, 32, 96, 64
        )
        self.maxpool_4a = MaxPool3dTFPadding(
            kernel_size=(3, 3, 3), stride=(2, 2, 2), padding="SAME"
        )
        self.mixed_4b = InceptionBlock(
            self.mixed_3c.output_dim, 192, 96, 208, 16, 48, 64
        )
        self.mixed_4c = InceptionBlock(
            self.mixed_4b.output_dim, 160, 112, 224, 24, 64, 64
        )
        self.mixed_4d = InceptionBlock(
            self.mixed_4c.output_dim, 128, 128, 256, 24, 64, 64
        )
        self.mixed_4e = InceptionBlock(
            self.mixed_4d.output_dim, 112, 144, 288, 32, 64, 64
        )
        self.mixed_4f = InceptionBlock(
            self.mixed_4e.output_dim, 256, 160, 320, 32, 128, 128
        )
        self.maxpool_5a = self.maxPool3d_5a_2x2 = MaxPool3dTFPadding(
            kernel_size=(2, 2, 2), stride=(2, 2, 2), padding="SAME"
        )
        self.mixed_5b = InceptionBlock(
            self.mixed_4f.output_dim, 256, 160, 320, 32, 128, 128
        )
        self.mixed_5c = InceptionBlock(
            self.mixed_5b.output_dim, 384, 192, 384, 48, 128, 128
        )
        self.fc = nn.Linear(self.mixed_5c.output_dim, num_classes)

    def forward(self, inputs):
        """Defines the S3DG base architecture."""
        # (bs, C, T, H, W) -> (bs, T, C, H, W)
        inputs = inputs.permute(0, 2, 1, 3, 4)

        net = self.conv1(inputs)
        net = self.maxpool_2a(net)
        net = self.conv_2b(net)
        net = self.conv_2c(net)
        if self.gating:
            net = self.gating(net)
        net = self.maxpool_3a(net)
        net = self.mixed_3b(net)
        net = self.mixed_3c(net)
        net = self.maxpool_4a(net)
        net = self.mixed_4b(net)
        net = self.mixed_4c(net)
        net = self.mixed_4d(net)
        net = self.mixed_4e(net)
        net = self.mixed_4f(net)
        net = self.maxpool_5a(net)
        net = self.mixed_5b(net)
        net = self.mixed_5c(net)

        net = torch.mean(net, dim=[2, 3, 4])

        return self.fc(net)

In [ ]:
# Example of how to use the model
model = S3D(num_classes=2)

# Check param
param = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Model has {param} parameters")

# Test the model with a random input (batch_size, channels, frames, height, width)
inputs = torch.rand(1, 3, 15, 224, 224)

output = model(inputs)

print(output.shape)

Model has 9129618 parameters
torch.Size([1, 2])


In [ ]:
del model, inputs, output

# Load dataset

In [ ]:
class VideoDataset(Dataset):
    def __init__(self, root_dir, phase="train", transform=None, n_frames=None):
        """
        Args:
            root_dir (string): Directory with all the videos (each video as a subdirectory of frames).
            transform (callable, optional): Optional transform to be applied on a sample.
            n_frames (int, optional): Number of frames to sample from each video, uniformly. If None, use all frames.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.n_frames = n_frames
        self.phase = phase
        self.videos, self.labels = self._load_videos()

    def _load_videos(self):
        videos, labels = [], []
        class_id = 0

        video_folders = os.listdir(os.path.join(self.root_dir, self.phase))

        for folder in video_folders:
            video_paths = os.listdir(os.path.join(self.root_dir, self.phase, folder))

            for video_path in video_paths:
                video_folder = os.path.join(
                    self.root_dir, self.phase, folder, video_path
                )
                frames = sorted(
                    (os.path.join(video_folder, f) for f in os.listdir(video_folder)),
                    key=lambda f: int(
                        "".join(filter(str.isdigit, os.path.basename(f)))
                    ),
                )

                if self.n_frames:
                    frames = self._uniform_sample(frames, self.n_frames)

                videos.append(frames)
                labels.append(class_id)

            class_id += 1

        return videos, labels

    def _uniform_sample(self, frames, n_frames):
        """
        Helper method to uniformly sample n_frames from the frames list.
        """
        stride = max(1, len(frames) // n_frames)
        sampled = [frames[i] for i in range(0, len(frames), stride)]
        return sampled[:n_frames]

    def __len__(self):
        return len(self.videos)

    def __getitem__(self, idx):
        video_frames = self.videos[idx]
        label = self.labels[idx]
        images = []
        for frame_path in video_frames:
            image = Image.open(frame_path).convert("RGB")
            if self.transform:
                image = self.transform(image)
            images.append(image)

        # Stack images along new dimension (sequence length)
        data = torch.stack(images, dim=0)

        # Rearrange to have the shape (C, T, H, W)
        data = data.permute(1, 0, 2, 3)
        return data, label

In [ ]:
BATCH_SIZE = 12
MAX_LEN = 15
IMAGE_SIZE = 224


transform = transforms.Compose(
    [
        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        transforms.ToTensor(),
    ]
)

# Load dataset
train_dataset = VideoDataset(
    root_dir="./dataset/rwf-2000", phase="train", transform=transform, n_frames=MAX_LEN
)

val_dataset = VideoDataset(
    root_dir="./dataset/rwf-2000", phase="val", transform=transform, n_frames=MAX_LEN
)

# Count number of cpus
cpus = os.cpu_count()
print(f"Number of cpus: {cpus}")

# Create data loaders
train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, num_workers=cpus, shuffle=True
)
val_loader = DataLoader(
    val_dataset, batch_size=BATCH_SIZE, num_workers=cpus, shuffle=False
)

# test
for data, label in train_loader:
    print(data.shape, label)
    break

Number of cpus: 80
torch.Size([12, 3, 15, 224, 224]) tensor([0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0])


# Train model

In [ ]:
def colorstr(*input):
    *args, string = input if len(input) > 1 else ("blue", "bold", input[0])
    colors = {
        "black": "\033[30m",  # basic colors
        "red": "\033[31m",
        "green": "\033[32m",
        "yellow": "\033[33m",
        "blue": "\033[34m",
        "magenta": "\033[35m",
        "cyan": "\033[36m",
        "white": "\033[37m",
        "bright_black": "\033[90m",  # bright colors
        "bright_red": "\033[91m",
        "bright_green": "\033[92m",
        "bright_yellow": "\033[93m",
        "bright_blue": "\033[94m",
        "bright_magenta": "\033[95m",
        "bright_cyan": "\033[96m",
        "bright_white": "\033[97m",
        "end": "\033[0m",  # misc
        "bold": "\033[1m",
        "underline": "\033[4m",
    }
    return "".join(colors[x] for x in args) + f"{string}" + colors["end"]

In [ ]:
def train_model(
    model, train_loader, val_loader, criterion, optimizer, num_epochs=25, device="cuda"
):
    """
    Function to train the model.

    Parameters:
    - model: The neural network model to train.
    - train_loader: DataLoader for the training set.
    - val_loader: DataLoader for the validation set.
    - criterion: The loss function.
    - optimizer: The optimization algorithm.
    - num_epochs: Number of epochs to train for.
    - device: The device to run the training on, 'cuda' or 'cpu'.

    Returns:
    - model: The trained model.
    """
    since = time.time()

    history = {
        "train_loss": [],
        "train_acc": [],
        "val_loss": [],
        "val_acc": [],
        "lr": [],
    }
    best_val_acc = 0.0

    # Send the model to the specified device
    model.to(device)

    # Loop over the dataset multiple times
    for epoch in range(num_epochs):
        LOGGER.info(colorstr(f"Epoch {epoch}/{num_epochs-1}:"))

        # Each epoch has a training and validation phase
        for phase in ["train", "val"]:
            if phase == "train":
                LOGGER.info(
                    colorstr("bright_yellow", "bold", "\n%20s" + "%15s" * 3)
                    % ("Training:", "gpu_mem", "loss", "acc")
                )
                model.train()
            else:
                LOGGER.info(
                    colorstr("bright_green", "bold", "\n%20s" + "%15s" * 3)
                    % ("Validation:", "gpu_mem", "loss", "acc")
                )
                model.eval()

            running_items = 0
            running_loss = 0.0
            running_corrects = 0

            # Use the appropriate data loader
            data_loader = train_loader if phase == "train" else val_loader

            _phase = tqdm(
                data_loader,
                total=len(data_loader),
                bar_format="{desc} {percentage:>7.0f}%|{bar:10}{r_bar}{bar:-10b}",
                unit="batch",
            )

            # Iterate over data.
            for inputs, labels in _phase:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward
                # Track history only in train
                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward + optimize only if in training phase
                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_items += outputs.size(0)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                epoch_loss = running_loss / running_items
                epoch_acc = running_corrects / running_items

                mem = f"{torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0:.3g}GB"
                desc = ("%35s" + "%15.6g" * 2) % (
                    mem,
                    epoch_loss,
                    epoch_acc,
                )
                _phase.set_description_str(desc)

            if phase == "train":
                history["train_loss"].append(epoch_loss)
                history["train_acc"].append(epoch_acc.item())
            else:
                history["val_loss"].append(epoch_loss)
                history["val_acc"].append(epoch_acc.item())
                if epoch_acc > best_val_acc:
                    best_val_acc = epoch_acc
                    history["best_epoch"] = epoch

                print(f"Best val Acc: {best_val_acc:4f}")

    time_elapsed = time.time() - since
    history["INFO"] = (
        "Training complete in {:.0f}h {:.0f}m {:.0f}s with {} epochs - Best val Acc: {:4f}".format(
            time_elapsed // 3600,
            time_elapsed % 3600 // 60,
            time_elapsed % 60,
            num_epochs,
            best_val_acc,
        )
    )

    return model

In [ ]:
# Example usage (assuming you have defined your criterion and optimizer):
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = S3D(num_classes=2)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

trained_model = train_model(
    model, train_loader, val_loader, criterion, optimizer, num_epochs=50, device=device
)

Epoch 0/49:

           Training:        gpu_mem           loss            acc
                             14.3GB       0.681484        0.58625     100%|██████████| 134/134 [00:53<00:00,  2.52batch/s]

         Validation:        gpu_mem           loss            acc
                             14.3GB       0.918393         0.5525     100%|██████████| 34/34 [00:08<00:00,  3.82batch/s]
Epoch 1/49:

           Training:        gpu_mem           loss            acc


Best val Acc: 0.552500


                             14.3GB       0.647335       0.629375     100%|██████████| 134/134 [00:53<00:00,  2.51batch/s]

         Validation:        gpu_mem           loss            acc
                             14.3GB       0.619618         0.6225     100%|██████████| 34/34 [00:08<00:00,  3.80batch/s]
Epoch 2/49:

           Training:        gpu_mem           loss            acc


Best val Acc: 0.622500


                             14.3GB       0.605212       0.674375     100%|██████████| 134/134 [00:53<00:00,  2.50batch/s]

         Validation:        gpu_mem           loss            acc
                             14.3GB       0.558487         0.7075     100%|██████████| 34/34 [00:09<00:00,  3.74batch/s]
Epoch 3/49:

           Training:        gpu_mem           loss            acc


...

Best val Acc: 0.787500


                             14.3GB       0.163983         0.9425     100%|██████████| 134/134 [01:01<00:00,  2.17batch/s]

         Validation:        gpu_mem           loss            acc
                             14.3GB       0.949909           0.74     100%|██████████| 34/34 [00:09<00:00,  3.68batch/s]
Epoch 49/49:

           Training:        gpu_mem           loss            acc


Best val Acc: 0.787500


                             14.3GB       0.122225       0.950625     100%|██████████| 134/134 [00:56<00:00,  2.36batch/s]

         Validation:        gpu_mem           loss            acc
                             14.3GB         1.1354           0.73     100%|██████████| 34/34 [00:09<00:00,  3.74batch/s]

Best val Acc: 0.787500
